In [1]:
from transformers import MambaConfig, MambaModel
from glob import glob
from torch import nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import json
from tqdm import tqdm
import copy
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

import dadam as optim
import cosine_lr_scheduler as scheduler

/home/lonian/anaconda3/envs/mamba_hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# delay token

In [51]:
import numpy as np

data = np.load('/mnt/gestalt/home/lonian/datasets/MusicBench/data_token/data_aug2/_0-2meOf9qY_2.npy', allow_pickle=True)

In [52]:
print(data.shape)

(4, 500)


In [53]:
print(data[:, :12])

[[ 166  166  166  166  166   83   83   83   83   83   83   83]
 [1931 1931 2044 2044 2044 1931 1931 1931 1931 1931 1931 1931]
 [2019 2019  924 2019 2019 2019 2019 2019 2019 2019 2019 2019]
 [1770 1755 1770 1854 1854 1770 1854 1854 1854 1854 1854 1854]]


In [44]:
z = torch.LongTensor([data])
z = z.view(1, -1)
print(z.shape)
print(z[:, :12])

torch.Size([1, 2000])
tensor([[166, 166, 166, 166, 166,  83,  83,  83,  83,  83,  83,  83]])


In [ ]:
z[:, :1]

In [49]:
torch.zeros_like(z[:, :1])

tensor([[0]])

In [48]:
z = torch.cat([z, torch.zeros_like(z[:, :1]) + 2048], dim=1)
# values = z[:, indexes.view(-1)]
# values = values.view(B, K, indexes.shape[-1])
print(z)

tensor([[ 166,  166,  166,  ..., 1387,  539, 2048]])


In [17]:
if data.shape[1] != 500:
    print('hi')
assert data.shape[1] == 500

## parallel to delay

In [54]:
K, L = data.shape
delay = np.zeros((K, L))+2048
for i in range(K):
    delay[i, i+1:] = data[i, :499-i]

In [55]:
print(delay[:, :10])
print(data[:, :10])

[[2048.  166.  166.  166.  166.  166.   83.   83.   83.   83.]
 [2048. 2048. 1931. 1931. 2044. 2044. 2044. 1931. 1931. 1931.]
 [2048. 2048. 2048. 2019. 2019.  924. 2019. 2019. 2019. 2019.]
 [2048. 2048. 2048. 2048. 1770. 1755. 1770. 1854. 1854. 1770.]]
[[ 166  166  166  166  166   83   83   83   83   83]
 [1931 1931 2044 2044 2044 1931 1931 1931 1931 1931]
 [2019 2019  924 2019 2019 2019 2019 2019 2019 2019]
 [1770 1755 1770 1854 1854 1770 1854 1854 1854 1854]]


In [56]:
print(delay[:, :10])
print(data[:, :10])

[[2048.  166.  166.  166.  166.  166.   83.   83.   83.   83.]
 [2048. 2048. 1931. 1931. 2044. 2044. 2044. 1931. 1931. 1931.]
 [2048. 2048. 2048. 2019. 2019.  924. 2019. 2019. 2019. 2019.]
 [2048. 2048. 2048. 2048. 1770. 1755. 1770. 1854. 1854. 1770.]]
[[ 166  166  166  166  166   83   83   83   83   83]
 [1931 1931 2044 2044 2044 1931 1931 1931 1931 1931]
 [2019 2019  924 2019 2019 2019 2019 2019 2019 2019]
 [1770 1755 1770 1854 1854 1770 1854 1854 1854 1854]]


In [59]:
print(delay.shape)

(4, 500)


In [69]:
m = torch.ne(torch.LongTensor(delay),2048)

In [70]:
m

tensor([[False,  True,  True,  ...,  True,  True,  True],
        [False, False,  True,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True]])

In [61]:
mask = torch.zeros(4, 500, dtype=torch.bool).numpy()
for i in range (K):
    mask[i, i+1:]=1

In [65]:
print(mask[:, :10])

[[False  True  True  True  True  True  True  True  True  True]
 [False False  True  True  True  True  True  True  True  True]
 [False False False  True  True  True  True  True  True  True]
 [False False False False  True  True  True  True  True  True]]


## delay to parallel

In [33]:
para = np.zeros((4, 500))
for i in range(4):
    para[i, :499-i] = delay[i, i+1:]

In [34]:
print(delay[:, :10])
print(para[:, :10])

[[   0.  166.  166.  166.  166.  166.   83.   83.   83.   83.]
 [   0.    0. 1931. 1931. 2044. 2044. 2044. 1931. 1931. 1931.]
 [   0.    0.    0. 2019. 2019.  924. 2019. 2019. 2019. 2019.]
 [   0.    0.    0.    0. 1770. 1755. 1770. 1854. 1854. 1770.]]
[[ 166.  166.  166.  166.  166.   83.   83.   83.   83.   83.]
 [1931. 1931. 2044. 2044. 2044. 1931. 1931. 1931. 1931. 1931.]
 [2019. 2019.  924. 2019. 2019. 2019. 2019. 2019. 2019. 2019.]
 [1770. 1755. 1770. 1854. 1854. 1770. 1854. 1854. 1854. 1854.]]


In [35]:
print(delay[:, -10:])
print(para[:, -10:])

[[ 643. 1417. 1454. 1193.  793.  530.  801. 1706.  648.  248.]
 [ 901. 1415.  911. 1985.  144. 1098.  780.  689. 1415. 1122.]
 [ 496. 1331.  970.  743.   20. 1904.  783.  909. 1169. 1051.]
 [2014. 1519.   11. 1334. 1158. 2034. 1568.   11.  628.   55.]]
[[1417. 1454. 1193.  793.  530.  801. 1706.  648.  248.    0.]
 [ 911. 1985.  144. 1098.  780.  689. 1415. 1122.    0.    0.]
 [ 743.   20. 1904.  783.  909. 1169. 1051.    0.    0.    0.]
 [1158. 2034. 1568.   11.  628.   55.    0.    0.    0.    0.]]


In [36]:
data = torch.LongTensor(np.pad(data,((0, 0), (0, 5)),'constant',constant_values=(0,2048)))

In [37]:
data[:, -10:]

tensor([[ 801, 1706,  648,  248,  248, 2048, 2048, 2048, 2048, 2048],
        [ 689, 1415, 1122,   62, 1122, 2048, 2048, 2048, 2048, 2048],
        [1169, 1051,  874,  953, 1919, 2048, 2048, 2048, 2048, 2048],
        [  55, 1730, 1650, 1387,  539, 2048, 2048, 2048, 2048, 2048]])

# Json loading

In [39]:
import json

with open('/mnt/gestalt/home/lonian/datasets/MusicBench/musicbench_train_simba.json') as f:
    metadata = json.load(f)

In [61]:
import os
import torch

def to_delay(para):
    '''
    INPUT:
    tokens: a k layers RVQ token
    1 5 9... (course layer)
    2 6 10...
    3 7 11...
    4 8 12... (fine layer)
    
    OUTPUT:
    delay: the delay pattern
    1 5 9... (course layer)
    0 2 6 10...
    0 0 3 7 11...
    0 0 0 4 8 12... (fine layer)
    '''
    K, L = para.shape
    delay = np.zeros((4, 500))
    for i in range(K):
        delay[i, i:] = para[i, :500-i]
    return delay

class MB_Dataset(object):
    '''
    MusicBench dataset
    '''
    def __init__(self, metadata, root_path, L = 500) -> None:
        self.meta = metadata
        self.root = root_path
        self.length = L
        self.number = len(self.meta)
        self.special_token_id = 2048
    
    def __getitem__(self, idx):
        path = os.path.join(self.root, self.meta[idx]['location'][:-4]+'.npy')
        description = self.meta[idx]['main_caption']
        data = np.load(path, allow_pickle=True)
        K, L = data.shape
        # data = torch.LongTensor(np.pad(data,((0, 0), (self.length-L, 0)),'constant',constant_values=(0,0)))
        data = torch.LongTensor(np.pad(data,((0, 0), (0, self.length-L)),'constant',constant_values=(0,self.special_token_id)))
        data = to_delay(data)
        # data = torch.LongTensor(data)
        # mask = torch.ne(torch.LongTensor(data), 2048)
        # return seq[:, 0:499], seq[:, 1:500]
        mask = (data == 2048)
        return torch.LongTensor(data[:, :-1]), torch.LongTensor(data[:, 1:]), description
    
    def __len__(self):
        return len(self.meta)

In [62]:
train_data = MB_Dataset(metadata, root_path = '/mnt/gestalt/home/lonian/datasets/MusicBench/data_token')

In [46]:
import numpy as np
for x, y in train_data:
    if 2048 in x:
        print(x)
        break

tensor([[ 890, 1194, 1660,  ..., 2048, 2048, 2048],
        [   0, 1735, 1341,  ..., 2048, 2048, 2048],
        [   0,    0, 1243,  ..., 2048, 2048, 2048],
        [   0,    0,    0,  ..., 2048, 2048, 2048]])


In [48]:
mask = (x != 2048)

In [50]:
mask[0]

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, 

In [32]:
import numpy as np
d = train_data[2][0]
K, L = d.shape
print(K, L)

4 499


In [93]:
for i in range(499):
    print(train_data[2][0][:, i])
    print(train_data[2][1][:, i])
    print('')

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 0, 0, 0])
tensor([0, 0, 0, 0])

tensor([0, 

In [1]:
def cal_torch_model_params(model):
    '''
    :param model:
    :return:
    '''
    # Find total parameters and trainable parameters
    total_params = sum(p.numel() for p in model.parameters())
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {'total_params': total_params/1000000, 'total_trainable_params': total_trainable_params/1000000}

In [24]:
from simba import Mmamba
model_config = {
        'layers':48,
        'vocab_size':2048,
        'd_model':512,
        'drop_p':0.2,
        'd_state':1024,
    }

music_model = Mmamba(
    layers = model_config['layers'], 
    vocab_size = model_config['vocab_size'], 
    d_model = model_config['d_model'], 
    drop_p = model_config['drop_p'], 
    d_state = model_config['d_state'])

In [25]:
cal_torch_model_params(music_model)

{'total_params': 185.90336, 'total_trainable_params': 185.90336}

# Flan T5 encoder

In [13]:
from transformers import AutoTokenizer, T5Tokenizer
from transformers import T5EncoderModel
device = 'cuda:1'
text_encoder_name = 'google/flan-t5-large'
tokenizer = T5Tokenizer.from_pretrained(text_encoder_name)
text_encoder = T5EncoderModel.from_pretrained(text_encoder_name)
text_encoder = text_encoder.to(device)

/home/lonian/anaconda3/envs/mamba_hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
prompt = ['This techno song features a synth lead playing the main melody. This is accompanied by programmed percussion playing a simple kick focused beat. The hi-hat is accented in an open position on the 3-and count of every bar. The synth plays the bass part with a voicing that sounds like a cello. This techno song can be played in a club. The chord sequence is Gm, A7, Eb, Bb, C, F, Gm. The beat counts to 2. The tempo of this song is 128.0 beats per minute. The key of this song is G minor.',
          'This relaxing song is perfect for a coffee shop setting. With no vocals, the electric guitar takes the lead, starting with a descending run before moving into an arpeggiated chord and double stop hammer-on to a higher note. A descending slide follows, along with a chord run while the simple percussion keeps time with rim shots. The bass plays a single note on the first count of each bar while the piano provides backing chords. All of this comes together in common time, with a beat of four that will create a laid-back atmosphere.',
          'abc 123 test abc 123 test abc 123 test abc 123 test abc 123 test abc 123 test', 
          'gggggg 888888888 8 8 h j k l']

In [65]:
import numpy as np
from tqdm import tqdm
max_value = []
min_value = []
for x, y, text in tqdm(train_data):
    # text=text.to()
    batch = tokenizer(
                text, padding=True, return_tensors="pt"
            )
    input_ids, attention_mask = batch.input_ids.to(device), batch.attention_mask.to(device)

    with torch.set_grad_enabled(False):
        text_embedding = text_encoder(
            input_ids=input_ids, attention_mask=attention_mask
        ).last_hidden_state
    max_value.append(text_embedding.max())
    min_value.append(text_embedding.min())
    # print(f"Max value in embeddings: {text_embedding.max()}")
    # print(f"Min value in embeddings: {text_embedding.min()}")

  0%|          | 0/52768 [00:00<?, ?it/s]

100%|██████████| 52768/52768 [44:42<00:00, 19.67it/s]  


In [69]:
print(max(max_value), min(max_value))
print(max(min_value), min(min_value))

tensor(0.7420, device='cuda:1') tensor(0.2948, device='cuda:1')
tensor(-0.2693, device='cuda:1') tensor(-0.6210, device='cuda:1')


In [71]:
import torch

device = text_encoder.device
batch = tokenizer(
    prompt, padding=True, return_tensors="pt"
)
input_ids, attention_mask = batch.input_ids.to(device), batch.attention_mask.to(device)

with torch.no_grad():
    encoder_hidden_states = text_encoder(
        input_ids=input_ids, attention_mask=attention_mask
    ).last_hidden_state

boolean_encoder_mask = (attention_mask == 0).to(device)

In [72]:
encoder_hidden_states

tensor([[[-8.3523e-03,  8.6555e-02,  8.6719e-02,  ..., -1.4685e-02,
           4.5399e-02, -6.6821e-02],
         [-1.4971e-01, -1.2987e-01, -1.0717e-01,  ...,  3.9089e-02,
           5.2967e-02, -7.5909e-02],
         [ 9.1772e-02, -2.7156e-02,  6.4160e-03,  ..., -2.7615e-02,
           2.5132e-01,  1.7656e-02],
         ...,
         [-1.9992e-01, -1.2682e-01, -3.4221e-02,  ...,  3.2654e-02,
          -1.5909e-01,  1.2429e-01],
         [ 6.1361e-02, -7.0698e-02,  1.8656e-02,  ..., -2.4162e-02,
           1.0555e-01, -7.5626e-02],
         [-5.5562e-02, -1.4670e-02,  1.5642e-02,  ...,  1.5967e-02,
           1.0352e-01, -7.2200e-02]],

        [[-1.4489e-02,  1.3645e-01,  7.9403e-02,  ...,  6.9213e-03,
           1.0345e-01, -7.1033e-02],
         [-6.0123e-02,  4.7248e-02,  1.3254e-03,  ...,  2.0407e-02,
          -4.3003e-02, -3.7244e-02],
         [ 5.1176e-02, -2.5561e-02,  5.5612e-02,  ..., -3.2872e-02,
           2.1540e-01,  4.7707e-02],
         ...,
         [-6.7759e-03, -2

In [17]:
encoder_hidden_states.device

device(type='cuda', index=1)

In [73]:
def random_zeroing(batch_tensor, batch_zero_prob=0.1, bit_zero_prob=0.05):
    """
    Args:
    - batch_tensor (torch.Tensor): 输入的0, 1向量，维度为(B, L)
    - batch_zero_prob (float): 每个batch全变成0的概率
    - bit_zero_prob (float): batch中的1变成0的概率
    
    Returns:
    - torch.Tensor: 经过随机置零后的向量
    """
    batch_size, seq_length = batch_tensor.shape

    # Step 1: 每个batch有10%的几率全变成0
    batch_mask = torch.bernoulli(torch.full((batch_size,), 1 - batch_zero_prob))
    print(batch_mask.bool())
    batch_mask = batch_mask.bool()
    batch_tensor[~batch_mask] = 0
    # print(batch_tensor)

    # Step 2: 对于剩下的batch中的1，以5%的几率变成0
    bit_mask = torch.bernoulli(torch.full_like(batch_tensor.float(), 1 - bit_zero_prob))
    bit_mask = bit_mask.bool()
    print(bit_mask)
    batch_tensor[batch_mask] = batch_tensor[batch_mask] * bit_mask[batch_mask]

    return batch_tensor

In [74]:
print("Original Batch Tensor:")
print(attention_mask)

# 进行随机置零
output_tensor = random_zeroing(attention_mask)
print("\nBatch Tensor After Random Zeroing:")
print(output_tensor)

Original Batch Tensor:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [81]:
condition = encoder_hidden_states * boolean_encoder_mask.unsqueeze(-1)

In [82]:
condition

tensor([[[-8.3523e-03,  8.6555e-02,  8.6719e-02,  ..., -1.4685e-02,
           4.5399e-02, -6.6821e-02],
         [-1.4971e-01, -1.2987e-01, -1.0717e-01,  ...,  3.9089e-02,
           5.2967e-02, -7.5909e-02],
         [ 9.1772e-02, -2.7156e-02,  6.4160e-03,  ..., -2.7615e-02,
           2.5132e-01,  1.7656e-02],
         ...,
         [-1.9992e-01, -1.2682e-01, -3.4221e-02,  ...,  3.2654e-02,
          -1.5909e-01,  1.2429e-01],
         [ 6.1361e-02, -7.0698e-02,  1.8656e-02,  ..., -2.4162e-02,
           1.0555e-01, -7.5626e-02],
         [-5.5562e-02, -1.4670e-02,  1.5642e-02,  ...,  1.5967e-02,
           1.0352e-01, -7.2200e-02]],

        [[-1.4489e-02,  1.3645e-01,  7.9403e-02,  ...,  6.9213e-03,
           1.0345e-01, -7.1033e-02],
         [-6.0123e-02,  4.7248e-02,  1.3254e-03,  ...,  2.0407e-02,
          -4.3003e-02, -3.7244e-02],
         [ 5.1176e-02, -2.5561e-02,  5.5612e-02,  ..., -3.2872e-02,
           2.1540e-01,  4.7707e-02],
         ...,
         [-0.0000e+00, -0

In [80]:
boolean_encoder_mask = (output_tensor == 1).to(device)
boolean_encoder_mask

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  

In [ ]:
boolean_encoder_mask

In [28]:
boolean_encoder_mask

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, F

In [28]:
print(encoder_hidden_states.shape, attention_mask.shape)

torch.Size([1, 133, 1024]) torch.Size([1, 133])


### x mask

In [35]:
import torch

def _generate_square_subsequent_mask(
        sz: int,
        device = torch.device('cpu'),
        dtype = torch.bool,
        pad_num = None
):
    r"""Generate a square causal mask for the sequence.

    The masked positions are filled with float('-inf'). Unmasked positions are filled with float(0.0).
    """
    mask = torch.triu(
        torch.full((sz, sz), float('-inf'), dtype=dtype, device=device),
        diagonal=1,
    )
    for i in range(1, pad_num+1):
        mask[-1*i, :] = True
    return mask

In [38]:
_generate_square_subsequent_mask(10, dtype=bool, pad_num=0)

tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False,  True,  True,  True],
        [False, False, False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False, False, False,  True],
        [False, False, False, False, False, False, False, False, False, False]])

In [9]:
def _make_causal_mask(
    input_ids_shape: torch.Size, dtype: torch.dtype, device: torch.device, past_key_values_length: int = 0
):
    """
    Make causal mask used for bi-directional self-attention.
    """
    bsz, tgt_len = input_ids_shape
    mask = torch.full((tgt_len, tgt_len), torch.tensor(torch.finfo(dtype).min, device=device), device=device)
    mask_cond = torch.arange(mask.size(-1), device=device)
    mask.masked_fill_(mask_cond < (mask_cond + 1).view(mask.size(-1), 1), 0)
    mask = mask.to(dtype)

    if past_key_values_length > 0:
        mask = torch.cat([torch.zeros(tgt_len, past_key_values_length, dtype=dtype, device=device), mask], dim=-1)
    return mask[None, None, :, :].expand(bsz, 1, tgt_len, tgt_len + past_key_values_length)

In [20]:
_make_causal_mask(input_ids_shape=(5, 499), dtype=bool, device='cpu', past_key_values_length=1)

TypeError: torch.finfo() requires a floating point input type. Use torch.iinfo to handle 'torch.finfo'

# simba model

In [1]:
a = NaN

NameError: name 'NaN' is not defined

In [1]:
from simba import Mmamba, Text_Mmamba
num = 3
layers = 24
self_atten_layers = []
for i in range(layers):
    if i%(layers//num) == 0:
        self_atten_layers.append(i)
print(self_atten_layers)
model = Text_Mmamba(
        layers = layers,
        vocab_size = 2048,
        d_model = 1024,
        drop_p = 0.2, d_state=512, d_conv=4, expand=2, num_heads=16, inner=False, self_atten_layers=self_atten_layers)
print(model)

/home/lonian/anaconda3/envs/mamba_hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0, 8, 16]
Text_Mmamba(
  (embedding): ModuleList(
    (0-3): 4 x ScaledEmbedding(2048, 1024)
  )
  (backbone): ModuleList(
    (0): Outer_Block(
      (mamba): M_CA_F_Block(
        (mamba): MambaLayer(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (drop_path): DropPath(drop_prob=0.200)
          (mamba): Mamba2(
            (in_proj): Linear(in_features=1024, out_features=5152, bias=False)
            (conv1d): Conv1d(3072, 3072, kernel_size=(4,), stride=(1,), padding=(3,), groups=3072)
            (act): SiLU()
            (norm): RMSNorm()
            (out_proj): Linear(in_features=2048, out_features=1024, bias=False)
          )
        )
        (cross_atten): MCALayer(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (drop_path): DropPath(drop_prob=0.200)
          (cross_atten): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
       

## musicgen

In [56]:
import random
entries = ['This techno song features a synth lead playing the main melody. This is accompanied by programmed percussion playing a simple kick focused beat. The hi-hat is accented in an open position on the 3-and count of every bar. The synth plays the bass part with a voicing that sounds like a cello. This techno song can be played in a club. The chord sequence is Gm, A7, Eb, Bb, C, F, Gm. The beat counts to 2. The tempo of this song is 128.0 beats per minute. The key of this song is G minor.', "This is a new age piece. There is a flute playing the main melody with a lot of staccato notes. The rhythmic background consists of a medium tempo electronic drum beat with percussive elements all over the spectrum. There is a playful atmosphere to the piece. This piece can be used in the soundtrack of a children's TV show or an advertisement jingle.", "The song is an instrumental. The song is in medium tempo with a classical guitar playing a lilting melody in accompaniment style. The song is emotional and romantic. The song is a romantic instrumental song. The chord sequence is Gm, F6, Ebm. The time signature is 4/4. This song is in Adagio. The key of this song is G minor.", "This techno song features a synth lead playing the main melody.This techno song features a synth lead playing the main melody.This techno song features a synth lead playing the main melody.This techno song features a synth lead playing the main melody.This techno song features a synth lead playing the main melody.This techno song features a synth lead playing the main melody.This techno song features a synth lead playing the main melody.This techno song features a synth lead playing the main melody.This techno song features a synth lead playing the main melody.This techno song features a synth lead playing the main melody."]
new_entries = []
for entry in entries:
    words = [word for word in entry.split(" ") if random.random() >= 0.3]
    new_entries.append(" ".join(words))
entries = new_entries
empty_idx = torch.LongTensor([i for i, xi in enumerate(entries) if xi == ""])

inputs = tokenizer(entries, return_tensors='pt', padding=True)
mask = inputs['attention_mask']
mask[empty_idx, :] = 0

In [64]:
embeds = text_encoder(**inputs).last_hidden_state

In [65]:
from torch import nn
# dim, output_dim = 1024, 1024
output_proj = nn.Linear(dim, output_dim)
mask = inputs['attention_mask']
# embeds = output_proj(embeds.to(output_proj.weight))
embeds = (embeds * mask.unsqueeze(-1))

In [66]:
embeds.shape

torch.Size([4, 95, 1024])

In [60]:
import typing as tp
a = dict({
            "genre": ["Rock", "Hip-hop"],
            "description": ["A rock song with a guitar solo", "A hip-hop verse"]
        })

In [62]:
from itertools import chain
for attribute, batch in chain(a.items()):
    print(attribute, batch)

genre ['Rock', 'Hip-hop']
description ['A rock song with a guitar solo', 'A hip-hop verse']


# move test set

In [67]:
import json

with open('/mnt/gestalt/home/lonian/datasets/MusicBench/MusicBench_test_A_simba.json') as f:
    metadata_A = json.load(f)
    
with open('/mnt/gestalt/home/lonian/datasets/MusicBench/MusicBench_test_B_simba.json') as f:
    metadata_B = json.load(f)

In [72]:
import shutil
import os

root = '/mnt/gestalt/home/lonian/datasets/MusicBench/datashare'

for i in metadata_A:
    print(i['location'])
    path = os.path.join(root, i['location'])
    print(path)
    shutil.copyfile(path, path.replace('datashare/data', 'test_A'))
    # break

for i in metadata_B:
    print(i['location'])
    path = os.path.join(root, i['location'])
    print(path)
    shutil.copyfile(path, path.replace('datashare/data', 'test_B'))

data/-0xzrMun0Rs.wav
/mnt/gestalt/home/lonian/datasets/MusicBench/datashare/data/-0xzrMun0Rs.wav
data/-Dtir74TiUM.wav
/mnt/gestalt/home/lonian/datasets/MusicBench/datashare/data/-Dtir74TiUM.wav
data/-JVWfFPBl84.wav
/mnt/gestalt/home/lonian/datasets/MusicBench/datashare/data/-JVWfFPBl84.wav
data/-XkbErI_7EU.wav
/mnt/gestalt/home/lonian/datasets/MusicBench/datashare/data/-XkbErI_7EU.wav
data/-o0ZtQIkM60.wav
/mnt/gestalt/home/lonian/datasets/MusicBench/datashare/data/-o0ZtQIkM60.wav
data/-w12aSkQ9No.wav
/mnt/gestalt/home/lonian/datasets/MusicBench/datashare/data/-w12aSkQ9No.wav
data/-w8maIWtnUk.wav
/mnt/gestalt/home/lonian/datasets/MusicBench/datashare/data/-w8maIWtnUk.wav
data/0-7PyzhzuYQ.wav
/mnt/gestalt/home/lonian/datasets/MusicBench/datashare/data/0-7PyzhzuYQ.wav
data/04NjXsLyCl4.wav
/mnt/gestalt/home/lonian/datasets/MusicBench/datashare/data/04NjXsLyCl4.wav
data/05OJDYeHLMc.wav
/mnt/gestalt/home/lonian/datasets/MusicBench/datashare/data/05OJDYeHLMc.wav
data/0ISHZQJdeSw.wav
/mnt/gest